# 🎯 The Trade Desk - Snowflake End-to-End Demo

## Data Ingestion (S3) | DMF Data Quality | Horizon Catalog | Tableau Lineage | Semantic Models

---

### 📋 Demo Overview

Welcome to this comprehensive demonstration of Snowflake's modern data platform capabilities, tailored specifically for **The Trade Desk's** advertising analytics use case.

| Feature | What You'll See |
|---------|-----------------|
| 🪣 **S3 Ingestion** | Storage Integration, External Stages, Snowpipe |
| 🔄 **Transformation** | Medallion Architecture, Dynamic Tables |
| ✅ **Data Quality (DMF)** | Built-in Data Metric Functions |
| 🔗 **Data Lineage** | End-to-End: S3 → Snowflake → Tableau |
| 🔐 **Auto-Tagging & Masking** | Classification profiles with tag-based masking |
| 📊 **Semantic Models** | Business-friendly metrics layer |
| 🔍 **Cortex Search** | AI-powered natural language campaign discovery |

---

### 🏗️ Architecture Overview

```
┌─────────────────────────────────────────────────────────────────────────────────────┐
│                          END-TO-END DATA FLOW                                        │
├─────────────────────────────────────────────────────────────────────────────────────┤
│                                                                                      │
│   ┌─────────┐     ┌─────────────┐     ┌─────────────┐     ┌─────────────┐     ┌────┐│
│   │   S3    │     │    RAW      │     │  STAGING    │     │ CONFORMED   │     │ BI ││
│   │ BUCKET  │────▶│   LAYER     │────▶│   LAYER     │────▶│   LAYER     │────▶│    ││
│   │         │     │  (Bronze)   │     │  (Silver)   │     │   (Gold)    │     │    ││
│   └─────────┘     └─────────────┘     └─────────────┘     └─────────────┘     └────┘│
│        │                │                   │                   │                   │
│   Snowpipe          DMF Checks          DMF Checks         Semantic               │
│   Auto-Ingest       • Null Rate         • Uniqueness       Model                  │
│                     • Freshness         • Integrity        • KPIs                 │
│                                                            • Tableau              │
│                                                                                      │
│   ═══════════════════════════════════════════════════════════════════════════════   │
│                           HORIZON CATALOG LAYER                                      │
│   • Data Lineage (automatic) • Data Quality Monitoring • Tags & Classification      │
│                                                                                      │
└─────────────────────────────────────────────────────────────────────────────────────┘
```

---

**⏱️ Estimated Demo Time:** 30 minutes  
**👤 Presenter:** Snowflake SE Team


---

# 🚀 PART 1: Environment Setup

### What We're Building

We'll create a complete advertising analytics platform with:
- **Dedicated warehouse** for compute isolation
- **Medallion architecture** (Bronze → Silver → Gold)
- **Role-based access control** for governance

> 💡 **Key Point:** Notice how we establish governance from the very beginning—not as an afterthought!


In [ ]:
-- 1.1 Set Admin Context and Create Resources
USE ROLE ACCOUNTADMIN;

-- Create Demo Warehouse with auto-suspend for cost efficiency
CREATE WAREHOUSE IF NOT EXISTS TTD_DEMO_WH
    WAREHOUSE_SIZE = 'SMALL'
    AUTO_SUSPEND = 60
    AUTO_RESUME = TRUE
    INITIALLY_SUSPENDED = TRUE
    COMMENT = 'The Trade Desk Demo - Advertising Analytics';

-- Create Demo Database with descriptive comment
CREATE DATABASE IF NOT EXISTS TTD_ADVERTISING_DEMO
    COMMENT = 'End-to-end demo for The Trade Desk - Advertising Analytics Platform';

USE DATABASE TTD_ADVERTISING_DEMO;

-- Create schemas for medallion architecture
CREATE SCHEMA IF NOT EXISTS RAW COMMENT = 'Bronze Layer - Raw ingested data from S3';
CREATE SCHEMA IF NOT EXISTS STAGING COMMENT = 'Silver Layer - Cleansed and validated data';
CREATE SCHEMA IF NOT EXISTS CONFORMED COMMENT = 'Gold Layer - Business-ready analytics';
CREATE SCHEMA IF NOT EXISTS GOVERNANCE COMMENT = 'Data governance objects, DMFs, and policies';


In [ ]:
-- 1.2 Create Demo Role and Set Context
CREATE ROLE IF NOT EXISTS TTD_DEMO_ROLE;
GRANT USAGE ON WAREHOUSE TTD_DEMO_WH TO ROLE TTD_DEMO_ROLE;
GRANT ALL ON DATABASE TTD_ADVERTISING_DEMO TO ROLE TTD_DEMO_ROLE;
GRANT ALL ON ALL SCHEMAS IN DATABASE TTD_ADVERTISING_DEMO TO ROLE TTD_DEMO_ROLE;
GRANT CREATE TABLE, CREATE VIEW, CREATE DYNAMIC TABLE ON ALL SCHEMAS IN DATABASE TTD_ADVERTISING_DEMO TO ROLE TTD_DEMO_ROLE;

SET user_name = CURRENT_USER();

GRANT ROLE TTD_DEMO_ROLE TO USER IDENTIFIER($user_name);


-- Set working context
USE ROLE TTD_DEMO_ROLE;
USE WAREHOUSE TTD_DEMO_WH;
USE SCHEMA TTD_ADVERTISING_DEMO.RAW;


---

# 🪣 PART 2: Data Ingestion from S3

### The Challenge

The Trade Desk processes **billions of ad impressions daily**. Data arrives continuously in S3 buckets and needs to land in Snowflake with:
- ⚡ **Low latency** (minutes, not hours)
- 📋 **Full audit trail** (source file, row number)
- 🔒 **Secure access** (no credentials in code)

### The Snowflake Solution

```
┌──────────────────────────────────────────────────────────────────────┐
│                     S3 INGESTION ARCHITECTURE                         │
├──────────────────────────────────────────────────────────────────────┤
│                                                                       │
│   ┌─────────────┐         ┌─────────────────┐        ┌─────────────┐ │
│   │     S3      │         │    STORAGE      │        │  EXTERNAL   │ │
│   │   BUCKET    │◀───────▶│  INTEGRATION    │◀──────▶│   STAGE     │ │
│   │             │   IAM   │ (Secure Bridge) │        │             │ │
│   └─────────────┘  Trust  └─────────────────┘        └──────┬──────┘ │
│                                                             │        │
│                           ┌─────────────────────────────────┼────┐   │
│                           │                                 ▼    │   │
│                           │   ┌───────────┐         ┌──────────┐ │   │
│                           │   │ SNOWPIPE  │────────▶│   RAW    │ │   │
│                           │   │ (Auto)    │         │  TABLES  │ │   │
│                           │   └───────────┘         └──────────┘ │   │
│                           │        or                            │   │
│                           │   ┌───────────┐         ┌──────────┐ │   │
│                           │   │ COPY INTO │────────▶│   RAW    │ │   │
│                           │   │ (Batch)   │         │  TABLES  │ │   │
│                           │   └───────────┘         └──────────┘ │   │
│                           └──────────────────────────────────────┘   │
│                                                                       │
└──────────────────────────────────────────────────────────────────────┘
```

> 💡 **Key Point:** Storage Integration provides a secure bridge to S3—credentials are managed by Snowflake's IAM trust relationship, not hardcoded!


### 2.1 Storage Integration (Production Setup)

> ⚠️ **Note:** The code below shows the production pattern. Uncomment when you have AWS configured.


In [ ]:
-- PRODUCTION S3 SETUP (uncomment and configure for your environment)
/*
USE ROLE ACCOUNTADMIN;

-- Create storage integration pointing to your S3 bucket
CREATE OR REPLACE STORAGE INTEGRATION ttd_s3_integration
    TYPE = EXTERNAL_STAGE
    STORAGE_PROVIDER = 'S3'
    ENABLED = TRUE
    STORAGE_AWS_ROLE_ARN = 'arn:aws:iam::<aws_account_id>:role/ttd-snowflake-role'
    STORAGE_ALLOWED_LOCATIONS = (
        's3://ttd-advertising-data/campaigns/',
        's3://ttd-advertising-data/impressions/',
        's3://ttd-advertising-data/clicks/',
        's3://ttd-advertising-data/conversions/'
    )
    COMMENT = 'Storage integration for TTD advertising data in S3';

-- Get the AWS IAM user ARN (needed for trust policy)
DESC STORAGE INTEGRATION ttd_s3_integration;
-- Note: Copy STORAGE_AWS_IAM_USER_ARN and STORAGE_AWS_EXTERNAL_ID to configure AWS
*/

-- For this demo, we'll use internal stages that mirror the S3 pattern
SELECT 'Storage Integration pattern demonstrated - using internal stages for demo' AS status;


### 2.2 File Formats & Stages

Different data types require different file formats. We support:
- **JSON** for campaign metadata (flexible schema)
- **CSV** for impression events (high volume, structured)
- **Parquet** for optimized columnar storage


In [ ]:
-- Create File Formats
CREATE OR REPLACE FILE FORMAT json_format
    TYPE = JSON
    STRIP_OUTER_ARRAY = TRUE
    COMMENT = 'JSON file format for campaign and bid data';

CREATE OR REPLACE FILE FORMAT csv_format
    TYPE = CSV
    FIELD_DELIMITER = ','
    SKIP_HEADER = 1
    NULL_IF = ('NULL', 'null', '')
    FIELD_OPTIONALLY_ENCLOSED_BY = '"'
    COMMENT = 'CSV file format for impression and event data';

CREATE OR REPLACE FILE FORMAT parquet_format
    TYPE = PARQUET
    COMPRESSION = SNAPPY
    COMMENT = 'Parquet for optimized columnar storage';

-- Create Stages (mirrors S3 bucket structure)
CREATE OR REPLACE STAGE stage_campaigns FILE_FORMAT = json_format COMMENT = 'Campaign JSON files';
CREATE OR REPLACE STAGE stage_impressions FILE_FORMAT = csv_format COMMENT = 'Impression CSV files';
CREATE OR REPLACE STAGE stage_clicks FILE_FORMAT = csv_format COMMENT = 'Click event files';
CREATE OR REPLACE STAGE stage_conversions FILE_FORMAT = csv_format COMMENT = 'Conversion event files';


### 2.3 Raw Tables (Bronze Layer)

Our raw tables capture everything from the source:
- **Full payload** (JSON stored as VARIANT)
- **Source tracking** (filename, row number)
- **Ingest metadata** (timestamp, batch ID)

> 🎯 **Best Practice:** Never transform data in the raw layer—preserve it exactly as received for audit and reprocessing!


In [ ]:
-- Create Raw Tables with full audit trail
CREATE OR REPLACE TABLE raw_campaigns (
    raw_payload         VARIANT,           -- Full JSON payload preserved
    source_filename     STRING,            -- Which file did this come from?
    source_row_number   NUMBER,            -- Which row in the file?
    ingest_timestamp    TIMESTAMP_LTZ DEFAULT CURRENT_TIMESTAMP(),
    ingest_batch_id     STRING DEFAULT UUID_STRING()
) COMMENT = 'Raw campaign metadata from S3 JSON files';

CREATE OR REPLACE TABLE raw_impressions (
    impression_id       STRING,
    campaign_id         STRING,
    advertiser_id       STRING,
    creative_id         STRING,
    placement_id        STRING,
    device_type         STRING,
    geo_country         STRING,
    geo_region          STRING,
    impression_ts       STRING,  -- Keep as string for data quality validation
    bid_price           STRING,
    win_price           STRING,
    source_filename     STRING,
    source_row_number   NUMBER,
    ingest_timestamp    TIMESTAMP_LTZ DEFAULT CURRENT_TIMESTAMP()
) COMMENT = 'Raw ad impressions - billions of events daily';

CREATE OR REPLACE TABLE raw_clicks (
    click_id            STRING,
    impression_id       STRING,
    campaign_id         STRING,
    click_ts            STRING,
    landing_page_url    STRING,
    device_type         STRING,
    source_filename     STRING,
    ingest_timestamp    TIMESTAMP_LTZ DEFAULT CURRENT_TIMESTAMP()
) COMMENT = 'Raw click events';

CREATE OR REPLACE TABLE raw_conversions (
    conversion_id       STRING,
    click_id            STRING,
    campaign_id         STRING,
    conversion_ts       STRING,
    conversion_type     STRING,
    conversion_value    STRING,
    source_filename     STRING,
    ingest_timestamp    TIMESTAMP_LTZ DEFAULT CURRENT_TIMESTAMP()
) COMMENT = 'Raw conversion/goal events';


### 2.4 Snowpipe: Continuous Auto-Ingestion

Snowpipe provides **serverless, event-driven** data loading. When files land in S3:

1. 📨 S3 sends event notification to SQS
2. ⚡ Snowpipe picks up the event automatically
3. 📥 Data lands in raw tables within minutes

```
┌─────────┐    Event    ┌─────────┐    Auto     ┌───────────┐
│   S3    │───────────▶│   SQS   │───────────▶│ SNOWPIPE  │───▶ Tables
│  File   │  Notification│  Queue  │   Trigger  │ (Serverless)│
└─────────┘            └─────────┘            └───────────┘
```

> 💰 **Cost Tip:** Snowpipe charges per-file, not per-compute-hour. Optimize by batching small files!


In [ ]:
-- SNOWPIPE DEFINITION (Production - requires S3 integration)
/*
CREATE OR REPLACE PIPE pipe_impressions
    AUTO_INGEST = TRUE
    COMMENT = 'Real-time impression ingestion from S3'
AS
COPY INTO raw_impressions (
    impression_id, campaign_id, advertiser_id, creative_id, placement_id,
    device_type, geo_country, geo_region, impression_ts, bid_price, win_price,
    source_filename, source_row_number
)
FROM (
    SELECT $1, $2, $3, $4, $5, $6, $7, $8, $9, $10, $11,
           METADATA$FILENAME, METADATA$FILE_ROW_NUMBER
    FROM @stage_s3_impressions
)
FILE_FORMAT = csv_format;

-- Check pipe status
SELECT SYSTEM$PIPE_STATUS('pipe_impressions');
*/

SELECT 'Snowpipe pattern demonstrated - see commented code above' AS status;


### 2.5 Load Sample Data

For this demo, we'll generate realistic advertising data that simulates what would arrive from S3.

| Data Type | Volume | Description |
|-----------|--------|-------------|
| Campaigns | 100 | Campaign configurations with targeting |
| Impressions | 50,000 | Ad view events across devices/geos |
| Clicks | ~1,000 | User engagement (~2% CTR) |
| Conversions | ~100 | Goal completions (~10% conversion) |


In [ ]:
-- Generate Campaign Data (JSON payloads)
INSERT INTO raw_campaigns (raw_payload, source_filename, source_row_number)
SELECT 
    PARSE_JSON('{
        "campaign_id": "CMP-' || LPAD(seq4()::STRING, 6, '0') || '",
        "campaign_name": "' || 
            CASE MOD(seq4(), 5) 
                WHEN 0 THEN 'Q1 Brand Awareness'
                WHEN 1 THEN 'Holiday Retargeting'
                WHEN 2 THEN 'CTV Premium Reach'
                WHEN 3 THEN 'Mobile App Install'
                ELSE 'Programmatic Display'
            END || ' - ' || seq4() || '",
        "advertiser_id": "ADV-' || LPAD(MOD(seq4(), 50)::STRING, 4, '0') || '",
        "advertiser_name": "' || CASE MOD(seq4(), 10)
                WHEN 0 THEN 'TechCorp Inc' WHEN 1 THEN 'RetailMax'
                WHEN 2 THEN 'AutoDrive Motors' WHEN 3 THEN 'FinanceFirst'
                WHEN 4 THEN 'HealthPlus' WHEN 5 THEN 'TravelEasy'
                WHEN 6 THEN 'FoodDelight' WHEN 7 THEN 'SportsPro'
                WHEN 8 THEN 'MediaStream' ELSE 'GreenEnergy Co' END || '",
        "budget_daily": ' || (UNIFORM(1000, 50000, RANDOM())::INT) || ',
        "budget_total": ' || (UNIFORM(50000, 500000, RANDOM())::INT) || ',
        "start_date": "' || DATEADD(DAY, -UNIFORM(1, 90, RANDOM()), CURRENT_DATE())::DATE || '",
        "end_date": "' || DATEADD(DAY, UNIFORM(30, 180, RANDOM()), CURRENT_DATE())::DATE || '",
        "targeting": {"geo": ["US", "CA", "UK"], "device_types": ["desktop", "mobile", "ctv"]},
        "kpi_goal": "' || CASE MOD(seq4(), 4) WHEN 0 THEN 'CPM' WHEN 1 THEN 'CPC' WHEN 2 THEN 'CPA' ELSE 'ROAS' END || '",
        "status": "' || CASE MOD(seq4(), 3) WHEN 1 THEN 'paused' ELSE 'active' END || '"
    }'),
    'campaigns_batch_' || CURRENT_DATE() || '.json',
    seq4()
FROM TABLE(GENERATOR(ROWCOUNT => 100));

SELECT '✅ Loaded ' || COUNT(*) || ' campaigns' AS status FROM raw_campaigns;


In [ ]:
-- Generate Impression Data (50K events)
INSERT INTO raw_impressions (impression_id, campaign_id, advertiser_id, creative_id, placement_id,
    device_type, geo_country, geo_region, impression_ts, bid_price, win_price, source_filename, source_row_number)
SELECT
    'IMP-' || UUID_STRING(),
    'CMP-' || LPAD(MOD(seq4(), 100)::STRING, 6, '0'),
    'ADV-' || LPAD(MOD(seq4(), 50)::STRING, 4, '0'),
    'CRE-' || LPAD(MOD(seq4(), 200)::STRING, 5, '0'),
    'PLC-' || LPAD(MOD(seq4(), 500)::STRING, 5, '0'),
    CASE MOD(seq4(), 4) WHEN 0 THEN 'desktop' WHEN 1 THEN 'mobile' WHEN 2 THEN 'tablet' ELSE 'ctv' END,
    CASE MOD(seq4(), 5) WHEN 0 THEN 'US' WHEN 1 THEN 'CA' WHEN 2 THEN 'UK' WHEN 3 THEN 'DE' ELSE 'FR' END,
    CASE MOD(seq4(), 4) WHEN 0 THEN 'California' WHEN 1 THEN 'New York' WHEN 2 THEN 'Texas' ELSE 'Florida' END,
    DATEADD(SECOND, -UNIFORM(1, 604800, RANDOM()), CURRENT_TIMESTAMP())::STRING,
    ROUND(UNIFORM(0.50, 15.00, RANDOM()), 4)::STRING,
    ROUND(UNIFORM(0.25, 12.00, RANDOM()), 4)::STRING,
    'impressions_' || CURRENT_DATE() || '.csv',
    seq4()
FROM TABLE(GENERATOR(ROWCOUNT => 50000));

-- Generate Clicks (~2% CTR)
INSERT INTO raw_clicks (click_id, impression_id, campaign_id, click_ts, landing_page_url, device_type, source_filename)
SELECT 'CLK-' || UUID_STRING(), impression_id, campaign_id,
    DATEADD(SECOND, UNIFORM(1, 30, RANDOM()), TRY_TO_TIMESTAMP(impression_ts))::STRING,
    'https://example.com/landing/' || campaign_id, device_type, 'clicks_' || CURRENT_DATE() || '.csv'
FROM raw_impressions WHERE UNIFORM(0, 100, RANDOM()) < 2 LIMIT 1000;

-- Generate Conversions (~10% of clicks)
INSERT INTO raw_conversions (conversion_id, click_id, campaign_id, conversion_ts, conversion_type, conversion_value, source_filename)
SELECT 'CNV-' || UUID_STRING(), click_id, campaign_id,
    DATEADD(MINUTE, UNIFORM(5, 1440, RANDOM()), TRY_TO_TIMESTAMP(click_ts))::STRING,
    CASE MOD(seq4(), 4) WHEN 0 THEN 'purchase' WHEN 1 THEN 'signup' WHEN 2 THEN 'lead' ELSE 'app_install' END,
    ROUND(UNIFORM(10, 500, RANDOM()), 2)::STRING, 'conversions_' || CURRENT_DATE() || '.csv'
FROM raw_clicks WHERE UNIFORM(0, 100, RANDOM()) < 10 LIMIT 100;


In [ ]:
-- Verify Data Load ✅
SELECT '📊 DATA INGESTION SUMMARY' AS report
UNION ALL SELECT '═══════════════════════════════'
UNION ALL SELECT 'Campaigns: ' || (SELECT COUNT(*)::STRING FROM raw_campaigns)
UNION ALL SELECT 'Impressions: ' || (SELECT COUNT(*)::STRING FROM raw_impressions)
UNION ALL SELECT 'Clicks: ' || (SELECT COUNT(*)::STRING FROM raw_clicks)
UNION ALL SELECT 'Conversions: ' || (SELECT COUNT(*)::STRING FROM raw_conversions);


---

# 🔄 PART 3: Data Transformation

### The Medallion Architecture

Our transformation follows the **Bronze → Silver → Gold** pattern:

```
┌─────────────────────────────────────────────────────────────────────────────┐
│                         MEDALLION ARCHITECTURE                               │
├─────────────────────────────────────────────────────────────────────────────┤
│                                                                              │
│  🥉 BRONZE (RAW)           🥈 SILVER (STAGING)        🥇 GOLD (CONFORMED)   │
│  ─────────────────         ──────────────────         ───────────────────   │
│  • Raw data as-is          • Type casting            • Aggregated metrics   │
│  • Full audit trail        • Null handling           • Business KPIs        │
│  • No transformation       • Deduplication           • Dimensional model    │
│  • JSON preserved          • Data quality flags      • Ready for BI         │
│                                                                              │
│  raw_campaigns      ──▶    stg_campaigns      ──▶    dim_campaigns         │
│  raw_impressions    ──▶    stg_impressions    ──▶    fact_performance      │
│  raw_clicks         ──▶    stg_clicks         ──▶    (Dynamic Table!)      │
│  raw_conversions    ──▶    stg_conversions                                  │
│                                                                              │
└─────────────────────────────────────────────────────────────────────────────┘
```

> 🚀 **Game Changer:** Dynamic Tables automatically refresh when upstream data changes—**no Airflow needed!**


In [ ]:
USE SCHEMA TTD_ADVERTISING_DEMO.STAGING;

-- 🥈 SILVER LAYER: Staging Views with Data Cleansing

-- Flatten JSON and apply proper data types
CREATE OR REPLACE VIEW stg_campaigns AS
SELECT
    raw_payload:campaign_id::STRING AS campaign_id,
    raw_payload:campaign_name::STRING AS campaign_name,
    raw_payload:advertiser_id::STRING AS advertiser_id,
    raw_payload:advertiser_name::STRING AS advertiser_name,
    raw_payload:budget_daily::NUMBER(12,2) AS budget_daily,
    raw_payload:budget_total::NUMBER(12,2) AS budget_total,
    TRY_TO_DATE(raw_payload:start_date::STRING) AS start_date,
    TRY_TO_DATE(raw_payload:end_date::STRING) AS end_date,
    raw_payload:targeting:geo::ARRAY AS geo_targeting,
    raw_payload:kpi_goal::STRING AS kpi_goal,
    UPPER(raw_payload:status::STRING) AS status,
    source_filename, ingest_timestamp, ingest_batch_id
FROM RAW.raw_campaigns
WHERE raw_payload:campaign_id IS NOT NULL;

-- Clean impressions with data quality flag
CREATE OR REPLACE VIEW stg_impressions AS
SELECT impression_id, campaign_id, advertiser_id, creative_id, placement_id,
    UPPER(device_type) AS device_type, UPPER(geo_country) AS geo_country, geo_region,
    TRY_TO_TIMESTAMP(impression_ts) AS impression_ts,
    TRY_TO_NUMBER(bid_price, 12, 4) AS bid_price,
    TRY_TO_NUMBER(win_price, 12, 4) AS win_price,
    -- Data quality flag: TRUE if all required fields are valid
    CASE WHEN TRY_TO_TIMESTAMP(impression_ts) IS NOT NULL 
          AND TRY_TO_NUMBER(win_price, 12, 4) IS NOT NULL 
          AND campaign_id IS NOT NULL THEN TRUE ELSE FALSE END AS is_valid,
    source_filename, ingest_timestamp
FROM RAW.raw_impressions WHERE impression_id IS NOT NULL;

-- Deduplicated clicks
CREATE OR REPLACE VIEW stg_clicks AS
SELECT click_id, impression_id, campaign_id, TRY_TO_TIMESTAMP(click_ts) AS click_ts,
    landing_page_url, UPPER(device_type) AS device_type, source_filename, ingest_timestamp,
    ROW_NUMBER() OVER (PARTITION BY click_id ORDER BY ingest_timestamp DESC) AS rn
FROM RAW.raw_clicks WHERE click_id IS NOT NULL
QUALIFY rn = 1;

-- Validated conversions
CREATE OR REPLACE VIEW stg_conversions AS
SELECT conversion_id, click_id, campaign_id, TRY_TO_TIMESTAMP(conversion_ts) AS conversion_ts,
    UPPER(conversion_type) AS conversion_type, TRY_TO_NUMBER(conversion_value, 12, 2) AS conversion_value,
    source_filename, ingest_timestamp
FROM RAW.raw_conversions 
WHERE conversion_id IS NOT NULL AND TRY_TO_NUMBER(conversion_value, 12, 2) >= 0;


### 3.2 Gold Layer with Dynamic Tables 🌟

**Dynamic Tables** are Snowflake's answer to complex transformation orchestration:

| Feature | Traditional ETL | Dynamic Tables |
|---------|-----------------|----------------|
| Scheduling | External (Airflow) | Built-in |
| Dependency Management | Manual DAG | Automatic |
| Incremental Processing | Complex | Automatic |
| Refresh Optimization | Custom code | Snowflake-managed |

> 💡 **TARGET_LAG** controls freshness: `1 minute` = near real-time, `1 hour` = cost-optimized


In [ ]:
USE SCHEMA TTD_ADVERTISING_DEMO.CONFORMED;

-- 🥇 GOLD LAYER: Campaign Dimension
CREATE OR REPLACE TABLE dim_campaigns AS
SELECT campaign_id, campaign_name, advertiser_id, advertiser_name,
    budget_daily, budget_total, start_date, end_date, geo_targeting, kpi_goal, status,
    CURRENT_TIMESTAMP() AS created_at
FROM STAGING.stg_campaigns;

-- 🥇 GOLD LAYER: Dynamic Table for Real-Time Campaign Performance
-- This automatically refreshes when upstream data changes!
CREATE OR REPLACE DYNAMIC TABLE fact_campaign_performance
    TARGET_LAG = '1 minute'  -- Near real-time freshness
    WAREHOUSE = TTD_DEMO_WH
AS
SELECT
    DATE_TRUNC('HOUR', i.impression_ts) AS hour_ts,
    DATE(i.impression_ts) AS date_key,
    i.campaign_id, i.advertiser_id, i.device_type, i.geo_country,
    
    -- 📊 Volume Metrics
    COUNT(DISTINCT i.impression_id) AS impressions,
    COUNT(DISTINCT c.click_id) AS clicks,
    COUNT(DISTINCT cv.conversion_id) AS conversions,
    
    -- 💰 Financial Metrics
    SUM(i.win_price) AS total_spend,
    SUM(cv.conversion_value) AS conversion_revenue,
    
    -- 📈 Performance KPIs (calculated)
    ROUND(COUNT(DISTINCT c.click_id) * 100.0 / NULLIF(COUNT(DISTINCT i.impression_id), 0), 4) AS ctr,
    ROUND(SUM(i.win_price) * 1000.0 / NULLIF(COUNT(DISTINCT i.impression_id), 0), 2) AS cpm,
    ROUND(SUM(i.win_price) / NULLIF(COUNT(DISTINCT c.click_id), 0), 2) AS cpc,
    ROUND(SUM(i.win_price) / NULLIF(COUNT(DISTINCT cv.conversion_id), 0), 2) AS cpa,
    ROUND(SUM(cv.conversion_value) / NULLIF(SUM(i.win_price), 0), 2) AS roas
    
FROM STAGING.stg_impressions i
LEFT JOIN STAGING.stg_clicks c ON i.impression_id = c.impression_id
LEFT JOIN STAGING.stg_conversions cv ON c.click_id = cv.click_id
WHERE i.is_valid = TRUE
GROUP BY 1, 2, 3, 4, 5, 6;


In [ ]:
-- Preview the Dynamic Table results 🎉
SELECT date_key, campaign_id, device_type, geo_country,
    impressions, clicks, conversions,
    ROUND(total_spend, 2) AS spend,
    ctr AS "CTR%", cpm AS "CPM$", roas AS "ROAS"
FROM fact_campaign_performance
ORDER BY date_key DESC, impressions DESC
LIMIT 10;


---

# ✅ PART 4: Data Metric Functions (DMF)

### Built-in Data Quality—No External Tools!

📚 **Documentation:** https://docs.snowflake.com/en/user-guide/data-quality-intro

Snowflake offers **two types of DMFs**:

| Type | Where | Use Case |
|------|-------|----------|
| **System DMFs** | `SNOWFLAKE.CORE.*` | Ready-to-use quality checks |
| **Custom DMFs** | Your schema | Business-specific rules |

### System DMFs Available (SNOWFLAKE.CORE)

```
┌─────────────────────────────────────────────────────────────────────────────┐
│                    BUILT-IN SYSTEM DMFs (No Setup Required!)                 │
├─────────────────────────────────────────────────────────────────────────────┤
│                                                                              │
│  📊 NULL_PERCENT      - % of NULL values (Completeness)                     │
│  🔢 DUPLICATE_COUNT   - Count of duplicate values (Uniqueness)              │
│  ⏰ FRESHNESS         - Time since last record (Timeliness)                 │
│  🔍 UNIQUE_COUNT      - Count of distinct values                            │
│                                                                              │
│  Usage: SELECT SNOWFLAKE.CORE.NULL_PERCENT(SELECT column FROM table);       │
│                                                                              │
└─────────────────────────────────────────────────────────────────────────────┘
```

> 🎯 **Why DMF?** External tools require data extraction, separate infrastructure, and manual scheduling. DMF runs **inside Snowflake** with zero data movement!


In [ ]:
USE ROLE ACCOUNTADMIN;
USE SCHEMA TTD_ADVERTISING_DEMO.GOVERNANCE;

-- ============================================================================
-- 4.1 SYSTEM DMFs - Use Immediately! (SNOWFLAKE.CORE)
-- ============================================================================

-- 📊 NULL_PERCENT: What % of impression_ids are null?
SELECT SNOWFLAKE.CORE.NULL_PERCENT(
    SELECT impression_id FROM TTD_ADVERTISING_DEMO.RAW.raw_impressions
) AS null_percent_impression_id;






In [ ]:
-- 🔢 DUPLICATE_COUNT: How many duplicate impression_ids?
SELECT SNOWFLAKE.CORE.DUPLICATE_COUNT(
    SELECT impression_id FROM TTD_ADVERTISING_DEMO.RAW.raw_impressions
) AS duplicate_impression_ids;


In [ ]:
-- 📈 FRESHNESS: How fresh is the data? (time since last record)
SELECT SNOWFLAKE.CORE.FRESHNESS(
    SELECT ingest_timestamp FROM TTD_ADVERTISING_DEMO.RAW.raw_impressions
) AS data_freshness;


In [ ]:
-- 🔍 UNIQUE_COUNT: Unique campaigns with impressions
SELECT SNOWFLAKE.CORE.UNIQUE_COUNT(
    SELECT campaign_id FROM TTD_ADVERTISING_DEMO.RAW.raw_impressions
) AS unique_campaigns;


In [ ]:
-- 📋 Combined Quality Summary
SELECT 
    COUNT(*) AS total_rows,
    SNOWFLAKE.CORE.NULL_PERCENT(SELECT impression_id FROM TTD_ADVERTISING_DEMO.RAW.raw_impressions) AS null_pct,
    SNOWFLAKE.CORE.DUPLICATE_COUNT(SELECT impression_id FROM TTD_ADVERTISING_DEMO.RAW.raw_impressions) AS duplicates,
    SNOWFLAKE.CORE.UNIQUE_COUNT(SELECT campaign_id FROM TTD_ADVERTISING_DEMO.RAW.raw_impressions) AS unique_campaigns
FROM TTD_ADVERTISING_DEMO.RAW.raw_impressions;


In [ ]:
-- ============================================================================
-- 4.2 CUSTOM DMF - For Business-Specific Rules
-- ============================================================================
-- Example: win_price should never exceed bid_price (auction anomaly check)

-- Step 2: Create CUSTOMDMF STRING types
CREATE OR REPLACE DATA METRIC FUNCTION 
TTD_ADVERTISING_DEMO.GOVERNANCE.dmf_invalid_auction_prices(
    auction_prices_t TABLE(bid_price STRING, win_price STRING)
)
RETURNS NUMBER
COMMENT = 'Count of records where win_price > bid_price (should be 0!)'
AS
$$
SELECT COUNT(*) 
FROM auction_prices_t 
WHERE TRY_TO_NUMBER(win_price) > TRY_TO_NUMBER(bid_price)
$$;

In [ ]:
-- ✅ Alternative: Attach DMF first, then set schedule separately
-- Attach System DMFs
ALTER TABLE TTD_ADVERTISING_DEMO.RAW.raw_impressions 
    ADD DATA METRIC FUNCTION SNOWFLAKE.CORE.NULL_PERCENT ON (impression_id);

ALTER TABLE TTD_ADVERTISING_DEMO.RAW.raw_impressions 
    ADD DATA METRIC FUNCTION SNOWFLAKE.CORE.DUPLICATE_COUNT ON (impression_id);

    -- Then set the schedule for the table
ALTER TABLE TTD_ADVERTISING_DEMO.RAW.raw_impressions 
    SET DATA_METRIC_SCHEDULE = 'TRIGGER_ON_CHANGES';


In [ ]:
SHOW DATA METRIC FUNCTIONS IN SCHEMA TTD_ADVERTISING_DEMO.GOVERNANCE;

In [ ]:
ALTER TABLE TTD_ADVERTISING_DEMO.RAW.raw_impressions 
    ADD DATA METRIC FUNCTION TTD_ADVERTISING_DEMO.GOVERNANCE.dmf_invalid_auction_prices 
    ON (bid_price, win_price);


In [ ]:
-- ============================================================================
-- 🧪 4.4 DEMO: Simulate Good vs Bad Records (DMF will trigger!)
-- ============================================================================
-- Business Rule: In ad auctions, you should NEVER pay more than you bid!
--   ✅ GOOD: bid_price = $5.00, win_price = $3.50 (paid less than bid)
--   ❌ BAD:  bid_price = $2.00, win_price = $5.00 (paid MORE than bid!)
-- ============================================================================

-- Step 1: Check current invalid count (should be 0 or low)
SELECT '📊 BEFORE INSERT - Invalid auction records:' AS step, 
       COUNT(*) AS invalid_count
FROM TTD_ADVERTISING_DEMO.RAW.raw_impressions
WHERE TRY_TO_NUMBER(win_price, 12, 4) > TRY_TO_NUMBER(bid_price, 12, 4);





-- (Optional) Cleanup test records for re-running the demo
-- DELETE FROM TTD_ADVERTISING_DEMO.RAW.raw_impressions WHERE impression_id LIKE 'TEST-%';


In [ ]:

-- Step 2: Insert a GOOD record (win_price < bid_price) ✅
INSERT INTO TTD_ADVERTISING_DEMO.RAW.raw_impressions 
    (impression_id, campaign_id, advertiser_id, device_type, geo_country, 
     impression_ts, bid_price, win_price, source_filename)
VALUES 
    ('TEST-GOOD-001', 'CMP-000001', 'ADV-0001', 'desktop', 'US', 
     CURRENT_TIMESTAMP()::STRING, '5.00', '3.50', 'dmf_test.csv');



In [ ]:
-- Step 3: Insert a BAD record (win_price > bid_price) ❌
-- This violates the auction rule - DMF will catch this!
INSERT INTO TTD_ADVERTISING_DEMO.RAW.raw_impressions 
    (impression_id, campaign_id, advertiser_id, device_type, geo_country, 
     impression_ts, bid_price, win_price, source_filename)
VALUES 
    ('TEST-BAD-001', 'CMP-000001', 'ADV-0001', 'mobile', 'US', 
     CURRENT_TIMESTAMP()::STRING, '2.00', '5.00', 'dmf_test.csv');  -- 🚨 win > bid!

Now see th invalid_count has increased by 1


In [ ]:

-- Step 4: Check invalid count AFTER insert (should increase by 1!)
SELECT '📊 AFTER INSERT - Invalid auction records:' AS step,
       COUNT(*) AS invalid_count
FROM TTD_ADVERTISING_DEMO.RAW.raw_impressions
WHERE TRY_TO_NUMBER(win_price, 12, 4) > TRY_TO_NUMBER(bid_price, 12, 4);

In [ ]:
-- Step 5: Show the bad record we just inserted
SELECT '🚨 Bad Record Found!' AS alert, impression_id, bid_price, win_price,
       'DMF will flag this in monitoring results!' AS note
FROM TTD_ADVERTISING_DEMO.RAW.raw_impressions
WHERE impression_id = 'TEST-BAD-001';

In [ ]:
INSERT INTO TTD_ADVERTISING_DEMO.RAW.raw_impressions (impression_id, campaign_id, advertiser_id, creative_id, placement_id,
    device_type, geo_country, geo_region, impression_ts, bid_price, win_price, source_filename, source_row_number)
SELECT
    'IMP-' || UUID_STRING(),
    'CMP-' || LPAD(MOD(seq4(), 100)::STRING, 6, '0'),
    'ADV-' || LPAD(MOD(seq4(), 50)::STRING, 4, '0'),
    'CRE-' || LPAD(MOD(seq4(), 200)::STRING, 5, '0'),
    'PLC-' || LPAD(MOD(seq4(), 500)::STRING, 5, '0'),
    CASE MOD(seq4(), 4) WHEN 0 THEN 'desktop' WHEN 1 THEN 'mobile' WHEN 2 THEN 'tablet' ELSE 'ctv' END,
    CASE MOD(seq4(), 5) WHEN 0 THEN 'US' WHEN 1 THEN 'CA' WHEN 2 THEN 'UK' WHEN 3 THEN 'DE' ELSE 'FR' END,
    CASE MOD(seq4(), 4) WHEN 0 THEN 'California' WHEN 1 THEN 'New York' WHEN 2 THEN 'Texas' ELSE 'Florida' END,
    DATEADD(SECOND, -UNIFORM(1, 604800, RANDOM()), CURRENT_TIMESTAMP())::STRING,
    ROUND(UNIFORM(0.50, 15.00, RANDOM()), 4)::STRING,
    ROUND(UNIFORM(0.25, 12.00, RANDOM()), 4)::STRING,
    'impressions_' || CURRENT_DATE() || '.csv',
    seq4()
FROM TABLE(GENERATOR(ROWCOUNT => 50000));

---

# 🔗 PART 5: Data Lineage & Tableau Integration

Snowflake's **Horizon Catalog** automatically tracks every object dependency—from S3 source to Tableau dashboard!

### What Gets Tracked Automatically:
- ✅ Object-to-object dependencies
- ✅ Column-level lineage
- ✅ Access history (who queried what)
- ✅ External tool queries (Tableau, Looker, etc.)

> 🎯 **Impact Analysis:** Before changing any table, know which dashboards will break!


In [ ]:
-- 5.1 Query Object Dependencies (Automatic Lineage)
SELECT
    referencing_object_name AS downstream_object,
    referencing_object_domain AS downstream_type,
    '←' AS direction,
    referenced_object_name AS upstream_object,
    referenced_object_domain AS upstream_type
FROM SNOWFLAKE.ACCOUNT_USAGE.OBJECT_DEPENDENCIES
WHERE referencing_database = 'TTD_ADVERTISING_DEMO'
  AND referenced_database = 'TTD_ADVERTISING_DEMO'
ORDER BY downstream_object;


In [ ]:
-- 5.2 Register External Lineage to Tableau Dashboards
CREATE OR REPLACE TABLE TTD_ADVERTISING_DEMO.GOVERNANCE.external_lineage_registry (
    source_object STRING, target_system STRING, target_asset_name STRING,
    target_asset_url STRING, owner_email STRING, description STRING
);

-- Register Tableau dashboard dependencies
INSERT INTO TTD_ADVERTISING_DEMO.GOVERNANCE.external_lineage_registry VALUES
('SEMANTIC_CAMPAIGN_METRICS', 'Tableau', 'Campaign Performance Dashboard',
 'https://tableau.thetradedesk.com/CampaignPerformance', 'analytics@ttd.com',
 'Executive KPI dashboard'),
('FACT_CAMPAIGN_PERFORMANCE', 'Tableau', 'Geographic Performance Map',
 'https://tableau.thetradedesk.com/GeoPerformance', 'analytics@ttd.com',
 'World map visualization'),
('DIM_CAMPAIGNS', 'Tableau', 'Advertiser Portfolio Summary',
 'https://tableau.thetradedesk.com/AdvertiserSummary', 'sales@ttd.com',
 'Campaign and budget overview');

-- View registered downstream assets
SELECT '📊 ' || target_asset_name AS tableau_dashboard, 
       '← ' || source_object AS snowflake_source,
       owner_email AS owner
FROM TTD_ADVERTISING_DEMO.GOVERNANCE.external_lineage_registry;


### 🔐 5.3 Tag-Based Classification with Auto Tagging

📚 **Documentation:** https://docs.snowflake.com/en/user-guide/classify-auto

Snowflake can **automatically discover and tag sensitive information**! Here's the flow:

```
┌─────────────────────────────────────────────────────────────────────────────┐
│                    AUTO-TAGGING CLASSIFICATION FLOW                          │
├─────────────────────────────────────────────────────────────────────────────┤
│                                                                              │
│   1. CREATE TAG          2. CREATE PROFILE       3. CLASSIFY TABLE          │
│   ─────────────          ────────────────        ────────────────           │
│   governance.pii    ──▶  classification     ──▶  SYSTEM$CLASSIFY()          │
│                          profile with            auto-tags columns          │
│                          auto_tag: true          with PII tag               │
│                                                                              │
│   4. CREATE MASKING      5. ATTACH TO TAG        6. QUERY = MASKED!         │
│   ─────────────────      ───────────────         ────────────────           │
│   mask_string_pii   ──▶  ALTER TAG pii SET  ──▶  Different roles see        │
│                          MASKING POLICY          different data             │
│                                                                              │
└─────────────────────────────────────────────────────────────────────────────┘
```

| Role | What They See |
|------|--------------|
| **ACCOUNTADMIN** | Full data (unmasked) |
| **TTD_DEMO_ROLE** | `****MASKED****` |
| **PUBLIC** | `****MASKED****` |

> 🛡️ **Key Value:** Once tagged, masking is automatic—no manual column-by-column work!


In [ ]:
USE ROLE ACCOUNTADMIN;

-- ============================================================================
-- Step 1: Create a PII Tag
-- ============================================================================
CREATE OR REPLACE TAG TTD_ADVERTISING_DEMO.GOVERNANCE.pii 
    COMMENT = 'Tag for Personally Identifiable Information';

-- ============================================================================
-- Step 2: Grant permissions for classification
-- ============================================================================
GRANT EXECUTE DATA METRIC FUNCTION ON ACCOUNT TO ROLE TTD_DEMO_ROLE;

-- ============================================================================
-- Step 3: Create Classification Profile with Auto-Tagging
-- ============================================================================
CREATE OR REPLACE SNOWFLAKE.DATA_PRIVACY.CLASSIFICATION_PROFILE
    TTD_ADVERTISING_DEMO.GOVERNANCE.ttd_classification_profile({
        'minimum_object_age_for_classification_days': 0,
        'maximum_classification_validity_days': 30,
        'auto_tag': true
    });

-- ============================================================================
-- Step 4: Set Tag Map - Auto-tag columns with these semantic categories as PII
-- ============================================================================
CALL TTD_ADVERTISING_DEMO.GOVERNANCE.ttd_classification_profile!SET_TAG_MAP({
    'column_tag_map': [{
        'tag_name': 'TTD_ADVERTISING_DEMO.GOVERNANCE.PII',
        'tag_value': 'sensitive',
        'semantic_categories': ['NAME', 'EMAIL', 'PHONE_NUMBER', 'US_POSTAL_CODE']
    }]
});

-- ============================================================================
-- Step 5: Classify the dim_campaigns table (auto-tags PII columns!)
-- ============================================================================
CALL SYSTEM$CLASSIFY('TTD_ADVERTISING_DEMO.CONFORMED.DIM_CAMPAIGNS', 
    'TTD_ADVERTISING_DEMO.GOVERNANCE.TTD_CLASSIFICATION_PROFILE');

-- View the auto-tagged columns
SELECT column_name, tag_name, tag_value
FROM TABLE(INFORMATION_SCHEMA.TAG_REFERENCES_ALL_COLUMNS(
    'TTD_ADVERTISING_DEMO.CONFORMED.DIM_CAMPAIGNS', 'TABLE'
))
WHERE tag_name = 'PII';

-- ============================================================================
-- Step 6: Create Masking Policy for PII strings
-- ============================================================================
CREATE OR REPLACE MASKING POLICY TTD_ADVERTISING_DEMO.GOVERNANCE.mask_pii_string 
AS (val STRING) RETURNS STRING ->
    CASE
        WHEN CURRENT_ROLE() IN ('ACCOUNTADMIN') THEN val
        ELSE '****MASKED****'
    END;

-- ============================================================================
-- Step 7: Attach Masking Policy to the PII Tag (applies to ALL tagged columns!)
-- ============================================================================
ALTER TAG TTD_ADVERTISING_DEMO.GOVERNANCE.pii 
    SET MASKING POLICY TTD_ADVERTISING_DEMO.GOVERNANCE.mask_pii_string;

SELECT '✅ Auto-tagging classification complete!' AS status;

-- ============================================================================
-- 🎭 DEMO: See Masking in Action!
-- ============================================================================

-- As ACCOUNTADMIN - see full data
USE ROLE ACCOUNTADMIN;
SELECT '👑 ACCOUNTADMIN View (Full Data):' AS role_view;
SELECT campaign_id, campaign_name, advertiser_name, status
FROM TTD_ADVERTISING_DEMO.CONFORMED.dim_campaigns LIMIT 3;

-- As TTD_DEMO_ROLE - see masked data
USE ROLE TTD_DEMO_ROLE;
SELECT '🔒 TTD_DEMO_ROLE View (Masked):' AS role_view;
SELECT campaign_id, campaign_name, advertiser_name, status
FROM TTD_ADVERTISING_DEMO.CONFORMED.dim_campaigns LIMIT 3;


---

# 📊 PART 6: Semantic Model

### Business-Friendly Metrics Layer

The **semantic layer** provides consistent metric definitions that business users can trust:

| Technical Column | Business Name | Why It Matters |
|-----------------|---------------|----------------|
| `ctr` | Click-Through Rate (%) | Ad effectiveness |
| `cpm` | Cost Per Mille ($) | Pricing efficiency |
| `roas` | Return on Ad Spend | ROI measurement |
| `cpa` | Cost Per Acquisition ($) | Conversion efficiency |

> 🎯 **CTR means the same thing everywhere**—no more "which CTR?" questions!


In [ ]:
USE ROLE TTD_DEMO_ROLE;
USE SCHEMA TTD_ADVERTISING_DEMO.CONFORMED;

-- Create Semantic Layer View with Business-Friendly Names
CREATE OR REPLACE SECURE VIEW semantic_campaign_metrics AS
SELECT
    -- Dimensions (business terminology)
    date_key AS "Date",
    fcp.campaign_id AS "Campaign ID",
    dc.campaign_name AS "Campaign Name",
    dc.advertiser_name AS "Advertiser",
    fcp.device_type AS "Device Type",
    fcp.geo_country AS "Country",
    dc.kpi_goal AS "KPI Goal",
    
    -- Core Metrics
    impressions AS "Impressions",
    clicks AS "Clicks",
    conversions AS "Conversions",
    ROUND(total_spend, 2) AS "Spend ($)",
    ROUND(conversion_revenue, 2) AS "Revenue ($)",
    
    -- Performance KPIs
    ROUND(ctr, 4) AS "Click-Through Rate (%)",
    ROUND(cpm, 2) AS "Cost Per Mille ($)",
    ROUND(cpc, 2) AS "Cost Per Click ($)",
    ROUND(cpa, 2) AS "Cost Per Acquisition ($)",
    ROUND(roas, 2) AS "Return on Ad Spend",
    
    -- Performance Tier
    CASE WHEN roas >= 3 THEN '🏆 High Performer'
         WHEN roas >= 1 THEN '✅ Profitable'
         WHEN roas > 0 THEN '⚠️ Below Target'
         ELSE '❌ No Conversions' END AS "Performance Tier"
FROM fact_campaign_performance fcp
JOIN dim_campaigns dc ON fcp.campaign_id = dc.campaign_id;


---

# 🎯 PART 7: Demo Queries

### Executive Dashboard - Ready for Tableau!


In [ ]:
-- 📈 Daily Performance Summary
SELECT "Date", SUM("Impressions") AS impressions, SUM("Clicks") AS clicks,
    SUM("Conversions") AS conversions, ROUND(SUM("Spend ($)"), 2) AS total_spend,
    ROUND(AVG("Click-Through Rate (%)"), 4) AS avg_ctr, ROUND(AVG("Return on Ad Spend"), 2) AS avg_roas
FROM semantic_campaign_metrics
GROUP BY "Date" ORDER BY "Date" DESC;


In [ ]:
-- 🏆 Top Performing Campaigns by ROAS
SELECT "Campaign Name", "Advertiser", SUM("Impressions") AS impressions,
    SUM("Clicks") AS clicks, ROUND(SUM("Spend ($)"), 2) AS spend,
    ROUND(SUM("Revenue ($)"), 2) AS revenue, ROUND(AVG("Return on Ad Spend"), 2) AS roas,
    MAX("Performance Tier") AS tier
FROM semantic_campaign_metrics
GROUP BY "Campaign Name", "Advertiser"
HAVING SUM("Conversions") > 0
ORDER BY roas DESC LIMIT 10;


---

# 🔍 PART 8: Cortex Search - AI-Powered Campaign Discovery

### The Problem
Business users shouldn't need to write SQL to find campaigns. They want to ask:
- *"Show me holiday retargeting campaigns"*
- *"Find high-budget CTV campaigns"*
- *"Which campaigns target automotive brands?"*

### The Solution: Cortex Search

```
┌─────────────────────────────────────────────────────────────────────┐
│                      CORTEX SEARCH FLOW                              │
├─────────────────────────────────────────────────────────────────────┤
│                                                                      │
│   User Query              Cortex Search              Results        │
│   ──────────              ─────────────              ───────        │
│   "Find retargeting  ───▶  AI Semantic   ───▶  Ranked Campaign     │
│    campaigns for           Matching +           List with          │
│    auto brands"            Embeddings           Relevance Scores   │
│                                                                      │
└─────────────────────────────────────────────────────────────────────┘
```

> 💡 **No SQL Required!** Business users search campaigns like they search Google.


In [ ]:
USE ROLE ACCOUNTADMIN;
USE SCHEMA TTD_ADVERTISING_DEMO.CONFORMED;

-- 8.1 Create Searchable Campaign Catalog with rich text descriptions
CREATE OR REPLACE TABLE campaign_search_catalog AS
SELECT campaign_id, campaign_name, advertiser_name, kpi_goal, status, budget_total,
    'Campaign: ' || campaign_name || '. Advertiser: ' || advertiser_name || 
    '. Goal: ' || kpi_goal || '. Status: ' || status ||
    '. Budget: $' || budget_total::STRING AS search_text
FROM dim_campaigns;

-- 8.2 Create Cortex Search Service
CREATE OR REPLACE CORTEX SEARCH SERVICE campaign_search_service
    ON search_text
    ATTRIBUTES campaign_id, campaign_name, advertiser_name, kpi_goal, status, budget_total
    WAREHOUSE = TTD_DEMO_WH
    TARGET_LAG = '1 hour'
    AS (SELECT * FROM campaign_search_catalog);

SELECT '✅ Cortex Search Service created!' AS status;


### 8.3 Search Campaigns with Natural Language 🎯

Try different queries to see semantic search in action:


In [ ]:
-- 🔍 Search: "brand awareness campaigns"
SELECT PARSE_JSON(
    SNOWFLAKE.CORTEX.SEARCH_PREVIEW(
        'TTD_ADVERTISING_DEMO.CONFORMED.CAMPAIGN_SEARCH_SERVICE',
        '{"query": "brand awareness campaigns with high budget", 
          "columns": ["campaign_name", "advertiser_name", "kpi_goal", "budget_total"], 
          "limit": 5}'
    )
):results AS brand_awareness_results;

-- 🔍 Search: "retargeting campaigns"
SELECT PARSE_JSON(
    SNOWFLAKE.CORTEX.SEARCH_PREVIEW(
        'TTD_ADVERTISING_DEMO.CONFORMED.CAMPAIGN_SEARCH_SERVICE',
        '{"query": "holiday retargeting campaigns", 
          "columns": ["campaign_name", "advertiser_name", "status"], 
          "limit": 5}'
    )
):results AS retargeting_results;

-- 🔍 Search: "mobile app campaigns"
SELECT PARSE_JSON(
    SNOWFLAKE.CORTEX.SEARCH_PREVIEW(
        'TTD_ADVERTISING_DEMO.CONFORMED.CAMPAIGN_SEARCH_SERVICE',
        '{"query": "mobile app install campaigns", 
          "columns": ["campaign_name", "advertiser_name", "kpi_goal"], 
          "limit": 5}'
    )
):results AS mobile_results;


### 8.4 Combine Search with Performance Analytics

Find campaigns by query, then show their actual performance metrics:


In [ ]:
-- Search for CTV campaigns, then show their performance
WITH search_results AS (
    SELECT value:campaign_id::STRING AS campaign_id
    FROM TABLE(FLATTEN(
        PARSE_JSON(
            SNOWFLAKE.CORTEX.SEARCH_PREVIEW(
                'TTD_ADVERTISING_DEMO.CONFORMED.CAMPAIGN_SEARCH_SERVICE',
                '{"query": "CTV premium campaigns", "columns": ["campaign_id"], "limit": 10}'
            )
        ):results
    ))
)
SELECT dc.campaign_name, dc.advertiser_name,
    SUM(fcp.impressions) AS impressions, SUM(fcp.clicks) AS clicks,
    ROUND(SUM(fcp.total_spend), 2) AS spend, ROUND(AVG(fcp.roas), 2) AS avg_roas
FROM search_results sr
JOIN dim_campaigns dc ON sr.campaign_id = dc.campaign_id
LEFT JOIN fact_campaign_performance fcp ON sr.campaign_id = fcp.campaign_id
GROUP BY 1, 2 ORDER BY avg_roas DESC NULLS LAST;

USE ROLE TTD_DEMO_ROLE;


In [ ]:
-- 📱 Device Performance Analysis
SELECT "Device Type", SUM("Impressions") AS impressions, SUM("Clicks") AS clicks,
    ROUND(AVG("Click-Through Rate (%)"), 4) AS avg_ctr,
    ROUND(SUM("Spend ($)"), 2) AS total_spend, ROUND(AVG("Cost Per Click ($)"), 2) AS avg_cpc
FROM semantic_campaign_metrics
GROUP BY "Device Type" ORDER BY impressions DESC;

-- 🌍 Geographic Performance
SELECT "Country", SUM("Impressions") AS impressions, SUM("Conversions") AS conversions,
    ROUND(SUM("Spend ($)"), 2) AS spend, ROUND(AVG("Return on Ad Spend"), 2) AS avg_roas
FROM semantic_campaign_metrics
GROUP BY "Country" ORDER BY spend DESC;


---

# 🎉 Demo Complete!

### Key Takeaways

| Capability | What We Demonstrated |
|------------|---------------------|
| 🪣 **S3 Ingestion** | Storage Integration, External Stages, Snowpipe pattern |
| 🔄 **Transformation** | Medallion Architecture, Dynamic Tables (no Airflow!) |
| ✅ **Data Quality** | DMF for null rate, duplicates, freshness |
| 🔗 **Data Lineage** | Automatic tracking S3 → Snowflake → Tableau |
| 🔐 **Auto-Tagging & Masking** | Classification profiles with tag-based masking |
| 📊 **Semantic Model** | Business-friendly metrics, consistent KPIs |
| 🔍 **Cortex Search** | AI-powered natural language campaign discovery |

---

### 🧹 Cleanup (Optional)

Run the cell below to remove all demo objects:

```sql
USE ROLE ACCOUNTADMIN;
DROP DATABASE IF EXISTS TTD_ADVERTISING_DEMO CASCADE;
DROP WAREHOUSE IF EXISTS TTD_DEMO_WH;
DROP ROLE IF EXISTS TTD_DEMO_ROLE;
```

---

**Thank you for joining this demo!** 🙏

For questions, reach out to your Snowflake account team.


---

# 🔗 PART 5: Horizon Catalog - Data Lineage & Tableau

### End-to-End Visibility: S3 → Snowflake → Tableau

Snowflake automatically tracks **every object dependency**:

```
┌─────────────────────────────────────────────────────────────────────────────────────┐
│                         END-TO-END DATA LINEAGE                                      │
├─────────────────────────────────────────────────────────────────────────────────────┤
│                                                                                      │
│  ┌─────────┐                    SNOWFLAKE                           ┌──────────┐   │
│  │   S3    │    ┌─────────┐   ┌─────────┐   ┌─────────┐            │ TABLEAU  │   │
│  │ BUCKET  │───▶│   RAW   │──▶│ STAGING │──▶│CONFORMED│───────────▶│DASHBOARD │   │
│  │         │    │ TABLES  │   │  VIEWS  │   │ TABLES  │            │          │   │
│  └─────────┘    └─────────┘   └─────────┘   └─────────┘            └──────────┘   │
│       │              │             │             │                       │        │
│       └──────────────┴─────────────┴─────────────┴───────────────────────┘        │
│                                    │                                              │
│                    ════════════════════════════════════                           │
│                           HORIZON CATALOG TRACKS:                                 │
│                    • Object Dependencies (automatic)                              │
│                    • Column-Level Lineage (automatic)                             │
│                    • External Tool Access (via ACCESS_HISTORY)                    │
│                    • Impact Analysis (query downstream)                           │
│                                                                                      │
└─────────────────────────────────────────────────────────────────────────────────────┘
```

> 💡 **Key Value:** Before changing any table, you can see exactly which dashboards will be affected!
